In [31]:
import pandas as pd

# Pfad zur Parquet-Datei
parquet_file_path = r'C:\Users\Tobias\Documents\BAconflictPrediction\ConflictPrediction\benchmark views\bm_cm_bootstrap_expanded_2021.parquet'

# Parquet-Datei in ein DataFrame einlesen
benchmark_data = pd.read_parquet(parquet_file_path)

benchmark_data

outcome
month_id country_id draw         
493      1          0           0
                    1           0
                    2           0
                    3           0
                    4          17
...                           ...
504      246        995         0
                    996        52
                    997         0
                    998         0
                    999         0

[2292000 rows x 1 columns]

In [10]:
import os
# create the feature- and actuals-data list
# set the feature and actuals year lists
feature_years = ['2017','2018','2019','2020']
actual_years = ['2018','2019','2020','2021']

actuals_df_list = []
features_df_list = []

# path to the current directory
current_dir = os.getcwd()

for i in range(len(feature_years)):
    # relative paths to the parquet files
    relative_path_features = os.path.join('..', 'data', 'cm_features_to_oct' + feature_years[i] + '.parquet')
    relative_path_actuals = os.path.join('..', 'data', 'cm_actuals_' + actual_years[i] + '.parquet')

    path_features = os.path.join(current_dir, relative_path_features)
    path_actuals = os.path.join(current_dir, relative_path_actuals)

    # append datasets to the lists
    actuals_df_list.append({'year':actual_years[i], 'data':pd.read_parquet(path_actuals, engine='pyarrow')})
    features_df_list.append({'year':feature_years[i], 'data':pd.read_parquet(path_features, engine='pyarrow')})

# concat the feature datasets, so that every data contains the observations starting with january 1990
for i in range(1,len(features_df_list)):
    features_df_list[i]['data'] = pd.concat([features_df_list[i-1]['data'], features_df_list[i]['data']])

country_list = sorted(features_df_list[3]['data'].index.get_level_values('country_id').unique().tolist())

# country group list of all four datasets
country_feature_group_list = []
country_actual_group_list = []
# fill list 
for i in range(len(features_df_list)):
    country_feature_group_list.append(features_df_list[i]['data'].groupby('country_id'))
    country_actual_group_list.append(actuals_df_list[i]['data'].groupby('country_id'))

In [11]:
from joblib import load
prediction_year = '2021' # 2019, 2020, 2021
dataset_index = actual_years.index(prediction_year)
actual_data = actuals_df_list[dataset_index]['data']
actuals_months = actual_data.index.get_level_values('month_id').unique()

vars = load('FinalNonzeroTask2_NN_2021_Hyperparamctr2204all.joblib')
NNet_prediction_list, country_list, pred_year_string, seed = vars

In [12]:
zero_fatalities_country_list = []
countries_with_high_percentage_list = []
someNonzero_fatalities_country_list = []

for country_id in country_list:
    feature_data = country_feature_group_list[0].get_group(country_id)

    # Berechnen des Prozentsatzes der Werte größer als 0 in der Spalte 'ged_sb'
    positive_percentage = (feature_data['ged_sb'] > 0).mean() * 100

    if (feature_data['ged_sb'] == 0).all():
        zero_fatalities_country_list.append(country_id)
    elif positive_percentage >= 60:
        countries_with_high_percentage_list.append(country_id)
    else:
        someNonzero_fatalities_country_list.append(country_id)

# Transform to dataframe

In [13]:
monthly_fatlities_pred_21 = pd.DataFrame()
# Iterieren Sie über die Daten und füllen Sie den DataFrame
for country in NNet_prediction_list:
    for s in country[pred_year_string][0]['s']:
        month_id = actuals_months[s-3]
        country_id = country['country_id']
        distribution = country[pred_year_string][0]['distribution'][s-3]
        draw = list(range(0,len(distribution)))

        index_tupel_list = []
        for i in range(len(distribution)):
            index_tupel_list.append((month_id,country_id,draw[i]))

        index = pd.MultiIndex.from_tuples(index_tupel_list, names=['month_id', 'country_id', 'draw'])

        # Erstelle eine Beispiel-Spalte "outcome"
        outcome = list(distribution)

        # Erstelle den DataFrame
        df = pd.DataFrame({'outcome': outcome}, index=index)

        monthly_fatlities_pred_21 = pd.concat([monthly_fatlities_pred_21, df], ignore_index=False)

In [14]:
monthly_fatlities_pred_21

outcome
month_id country_id draw         
493      1          0           0
                    1           0
                    2           0
                    3           0
                    4           0
...                           ...
504      246        195       247
                    196       277
                    197       302
                    198       327
                    199       335

[458400 rows x 1 columns]

In [25]:
import CRPS.CRPS as pscore
import numpy as np


country_list = sorted(monthly_fatlities_pred_21.index.get_level_values('country_id').unique().tolist())
# country group list of all four datasets
pred_21_country_group = monthly_fatlities_pred_21.groupby('country_id')
actual_21_group = actual_data.groupby('country_id')

crps_values_test = []

for country in pred_21_country_group:
    country_id = country[0]

    if country_id not in zero_fatalities_country_list:
        country_crps_list = []
        for i in range(0,12):
            y_true = actual_21_group.get_group(country_id).iloc[i,0]
            month = actuals_months[i]
            prediction = pred_21_country_group.get_group(country_id).xs(month, level='month_id').values.flatten()
            crps = pscore(prediction,y_true).compute()[0]
            country_crps_list.append(crps)
        crps_values_test.append(np.mean(country_crps_list))

mean_crps_test = np.mean(crps_values_test)


print('Mean CRPS = ' + str(mean_crps_test) + ' \\'+'\\')
print('')

    

Mean CRPS = 58.656955141843966 \\



In [33]:
import CRPS.CRPS as pscore
import numpy as np

bench_21_country_group = benchmark_data.groupby('country_id')
actual_21_group = actual_data.groupby('country_id')

crps_values_test = []

for country in bench_21_country_group:
    country_id = country[0]

    if country_id not in zero_fatalities_country_list:
        country_crps_list = []
        for i in range(0,12):
            y_true = actual_21_group.get_group(country_id).iloc[i,0]
            month = actuals_months[i]
            prediction = bench_21_country_group.get_group(country_id).xs(month, level='month_id').values.flatten()
            crps = pscore(prediction,y_true).compute()[0]
            country_crps_list.append(crps)
        crps_values_test.append(np.mean(country_crps_list))

mean_crps_test = np.mean(crps_values_test)


print('Mean CRPS = ' + str(mean_crps_test) + ' \\'+'\\')
print('')

Mean CRPS = 72.41006815868793 \\

